In [2]:
import numpy as np
import h5py
import pandas as pd
from itertools import islice
from collections import Counter
import re
from itertools import islice
from collections import Counter
from io import BytesIO

In [134]:
mouse = h5py.File('mouse_matrix_v11.h5', "r") # fot reading h5 dataframe
data=mouse.keys()
for key in mouse.keys():
    print(key)
for key in mouse['meta'].keys():
    print(key)
list_of_columns=mouse['meta']['genes'].keys()
print(list_of_columns)

data
meta
genes
info
samples
<KeysViewHDF5 ['chromosome', 'ensembl_gene_id', 'gene_biotype', 'gene_symbol', 'genes', 'start_position']>


In [254]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
    for key in f["meta/genes"].keys():
        a = f["meta/genes/"][key]
        print(key)
        print(a[:])

chromosome
[b'missing' b'11' b'11' ... b'MT' b'MT' b'MT']
ensembl_gene_id
[b'missing' b'ENSMUSG00000007777' b'ENSMUSG00000043644' ...
 b'ENSMUSG00000065947' b'ENSMUSG00000064367' b'ENSMUSG00000064368']
gene_biotype
[b'missing' b'protein_coding' b'lncRNA' ... b'protein_coding'
 b'protein_coding' b'protein_coding']
gene_symbol
[b'0610007P14Rik' b'0610009B22Rik' b'0610009L18Rik' ... b'mt-Nd4l'
 b'mt-Nd5' b'mt-Nd6']
genes
[b'0610007P14Rik' b'0610009B22Rik' b'0610009L18Rik' ... b'mt-Nd4l'
 b'mt-Nd5' b'mt-Nd6']
start_position
[b'missing' b'51576213' b'120239504' ... b'9877' b'11742' b'13552']


Now every dataset from h5 file we transform into single table in pandas and then concatenate them into one table
import h5py
arch_file = "/home/clever/test_cache/counts/archs4/mouse_matrix_v11.h5"
with h5py.File(arch_file, 'r') as arch_7:
    for key in arch_7["meta/samples"].keys():
        a = arch_7["meta/samples/"][key]
        
        print(key)
        print(a[:])

In [301]:
title=mouse['meta']['samples']['title']
title_data=pd.DataFrame(title, columns=['title'])
type=mouse['meta']['samples']['type']
type_data=pd.DataFrame(type, columns=['type'])
geo_accession=mouse['meta']['samples']['geo_accession']
geo_accession_data=pd.DataFrame(geo_accession, columns=['geo_accession'])
for col, dtype in geo_accession_data.dtypes.items():
    if dtype == np.object_:  # Only process byte object columns.
        geo_accession_data[col] = geo_accession_data[col].apply(lambda x: x.decode("utf-8"))
data_processing=mouse['meta']['samples']['data_processing']
data_processing_data=pd.DataFrame(data_processing, columns=['data_processing'])
characteristics_ch1=mouse['meta']['samples']['characteristics_ch1']
characteristics_ch1_data=pd.DataFrame(characteristics_ch1, columns=['characteristics_ch1'])
extract_protocol_ch1=mouse['meta']['samples']['extract_protocol_ch1']
extract_protocol_ch1_data=pd.DataFrame(extract_protocol_ch1, columns=['extract_protocol_ch1'])
instrument_model=mouse['meta']['samples']['instrument_model']
instrument_model_data=pd.DataFrame(instrument_model, columns=['instrument_model'])
library_selection=mouse['meta']['samples']['library_selection']
library_selection_data=pd.DataFrame(library_selection, columns=['library_selection'])
library_source=mouse['meta']['samples']['library_source']
library_source_data=pd.DataFrame(library_source, columns=['library_source'])
library_strategy=mouse['meta']['samples']['library_strategy']
library_strategy_data=pd.DataFrame(library_strategy, columns=['library_strategy'])
molecule_ch1=mouse['meta']['samples']['molecule_ch1']
molecule_ch1_data=pd.DataFrame(molecule_ch1, columns=['molecule_ch1'])
organism_ch1=mouse['meta']['samples']['organism_ch1']
organism_ch1_data=pd.DataFrame(organism_ch1, columns=['organism_ch1'])
platform_id=mouse['meta']['samples']['platform_id']
platform_id_data=pd.DataFrame(platform_id, columns=['platform_id'])
for col, dtype in platform_id_data.dtypes.items():
    if dtype == np.object_:  # Only process byte object columns.
        platform_id_data[col] = platform_id_data[col].apply(lambda x: x.decode("utf-8"))
readsaligned=mouse['meta']['samples']['readsaligned']
readsaligned_data=pd.DataFrame(readsaligned, columns=['readsaligned'])
readstotal=mouse['meta']['samples']['readstotal']
readstotal_data=pd.DataFrame(readstotal, columns=['readstotal'])
relation=mouse['meta']['samples']['relation']
relation_data=pd.DataFrame(relation, columns=['relation'])
series_id=mouse['meta']['samples']['series_id']
series_id_data=pd.DataFrame(series_id, columns=['series_id'])
source_name_ch1=mouse['meta']['samples']['source_name_ch1']
source_name_ch1_data=pd.DataFrame(source_name_ch1, columns=['source_name_ch1'])
#for col, dtype in source_name_ch1_data.dtypes.items():
#   if dtype == np.object_:  # Only process byte object columns.
#        source_name_ch1_data[col] = source_name_ch1_data[col].apply(lambda x: x.decode("utf-8"))
status=mouse['meta']['samples']['status']
status_data=pd.DataFrame(status, columns=['status'])
taxid_ch1=mouse['meta']['samples']['taxid_ch1']
taxid_ch1_data=pd.DataFrame(taxid_ch1, columns=['taxid_ch1'])
table=pd.concat([taxid_ch1_data,status_data,source_name_ch1_data,series_id_data,relation_data,readstotal_data,readsaligned_data,platform_id_data,organism_ch1_data, molecule_ch1_data,library_strategy_data,library_selection_data, library_source_data,instrument_model_data,extract_protocol_ch1_data, characteristics_ch1_data,data_processing_data,geo_accession_data,type_data,title_data ],sort=False, axis=1)
table.to_csv('table.csv')

### Декодируем все строки из юникод в обычные (убираем b' ')

for col, dtype in table.dtypes.items():
    if dtype == np.object_:  # Only process byte object columns.
        table[col] = table[col].apply(lambda x: x.decode("utf-8"))
table.to_csv('table.csv')

In [302]:
print(table.head())

  taxid_ch1                    status     source_name_ch1  \
0  b'10090'  b'Public on Apr 07 2015'    b'kidney cortex'   
1  b'10090'  b'Public on Jun 10 2016'  b'corpus callosum'   
2  b'10090'  b'Public on Jun 10 2016'      b'dorsal horn'   
3  b'10090'  b'Public on Jan 01 2016'      b'Lung tissue'   
4  b'10090'  b'Public on Sep 10 2012'           b'Cortex'   

                         series_id  \
0                      b'GSE54390'   
1                      b'GSE75330'   
2                      b'GSE75330'   
3                      b'GSE64027'   
4  b'GSE36025\tGSE49417\tGSE49847'   

                                            relation  readstotal  \
0  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    39116064   
1  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...     1136541   
2  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...      514285   
3  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    26804780   
4  b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...   265978941

In [269]:
print(geo_accession[28].decode("utf-8"))

GSM1849922


Let's delete rows with words - single cell in a column 'source_name_ch1'

In [303]:
df = pd.read_csv('table.csv') # read table into pandas dataframe
discard=["single cell"]
del_single=df[~df.source_name_ch1.str.contains('|'.join(discard))] # ~ - все, кроме того что дальше - после нее
del_single.to_csv('del_single.csv')

In [304]:
print(len(df))
print(table.head())

501054
  taxid_ch1                    status     source_name_ch1  \
0  b'10090'  b'Public on Apr 07 2015'    b'kidney cortex'   
1  b'10090'  b'Public on Jun 10 2016'  b'corpus callosum'   
2  b'10090'  b'Public on Jun 10 2016'      b'dorsal horn'   
3  b'10090'  b'Public on Jan 01 2016'      b'Lung tissue'   
4  b'10090'  b'Public on Sep 10 2012'           b'Cortex'   

                         series_id  \
0                      b'GSE54390'   
1                      b'GSE75330'   
2                      b'GSE75330'   
3                      b'GSE64027'   
4  b'GSE36025\tGSE49417\tGSE49847'   

                                            relation  readstotal  \
0  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    39116064   
1  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...     1136541   
2  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...      514285   
3  b'BioSample: https://www.ncbi.nlm.nih.gov/bios...    26804780   
4  b'Reanalyzed by: GSE80797\tSuperseded by: GSE9...   26

Create a table with only immune cells

In [305]:
df=pd.read_csv('del_single.csv')
immune_cell= df[df['source_name_ch1'].str.contains('Innate lymphoid cell|CD8|CD4|macrophage|microglia|Th17|T cell|B cell', case=False)] # можно усложнить до регулярного выражения !

immune_cell.to_csv('immune_cell.csv')

Create a table with uniqe values by cell type

In [306]:
df = pd.read_csv('del_single.csv')
cell_count=df.groupby('source_name_ch1')['platform_id'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
cell_count.to_csv('cell_count.csv') # put table into file .csv

Create a table with uniqe values by platform id

In [307]:
df = pd.read_csv('del_single.csv')
platform_count=df.groupby('platform_id')['source_name_ch1'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
platform_count.to_csv('platform_count.csv') # put table into file .csv

Create a table with uniqe values by cell type - only immune cells

In [308]:
df = pd.read_csv('immune_cell.csv')
cell_count=df.groupby('source_name_ch1')['platform_id'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
cell_count.to_csv('cell_count2.csv') # put table into file .csv

Create a table with uniqe values by platform id - only immune cells

In [309]:
df = pd.read_csv('immune_cell.csv')
platform_count=df.groupby('platform_id')['source_name_ch1'].value_counts() #count on two column basis
#cell_count=df['source_name_ch1'].value_counts() #create a table with count of uniq cell types
platform_count.to_csv('platform_count2.csv') # put table into file .csv

Let's change the last table. Remove very small values, type of cells into rows, platform id into columns


In [310]:
df= pd.read_csv('platform_count2.csv')
df.columns=['platform_id', 'source_name_ch1', 'counter'] # rename the columns heads
df=df[df.counter>10] # delete the values less than 10
table = pd.pivot_table(df, values='counter', index=['source_name_ch1'], columns=['platform_id'], aggfunc=np.sum) #create a table with platfom_id in columns
table.to_csv('platform_count3.csv') # put table into file .csv

In [14]:
df = pd.read_csv('del_single.csv')


Create a table with needing columns - макрофаги
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0) - сложение таблиц

In [15]:
wf = df[['geo_accession', 'platform_id', 'source_name_ch1']] #wf - working frame
macrophage= wf[wf['source_name_ch1'].str.contains('macrophage|macrophages', case=False)]
macrophage.insert(2, "Cell_type", 'macrophage')
conditions = [
    (macrophage['source_name_ch1'].str.contains('[B,m]one[-,\s]marrow', case=False, regex=True)),
    (macrophage['source_name_ch1'].str.contains('Peritoneal', case=False)),
    (macrophage['source_name_ch1'].str.contains('Alveolar|lung', case=False)),
    (macrophage['source_name_ch1'].str.contains('tumor|tumour', case=False)),
    (macrophage['source_name_ch1'].str.contains('Splenic', case=False)),
    (macrophage['source_name_ch1'].str.contains('hepatic', case=False)),
    (macrophage['source_name_ch1'].str.contains('sciatic nerve', case=False)),
    (macrophage['source_name_ch1'].str.contains('Phagocytic', case=False)),
    (macrophage['source_name_ch1'].str.contains('Peripheral', case=False)),
    (macrophage['source_name_ch1'].str.contains('muscle', case=False)),
    (macrophage['source_name_ch1'].str.contains('1 day after TBI', case=False)),
    (~macrophage['source_name_ch1'].str.contains('[B,m]one[-,\s]marrow|Peritoneal|Alveolar|tumor|tumour|Splenic|lung|hepatic|sciatic nerve|Phagocytic|Peripheral|muscle|1 day after TBI' , case=False))
]

values = ['Bone Marrow Derived macrophages', 'Peritoneal macrophage', 'Alveolar macrophage', 'tumor macrophage','Splenic macrophages','hepatic macrophages', 'sciatic nerve macrophages','Phagocytic macrophages', 'Peripheral macrophages','muscle macrophages','1 day after TBI macrophages', 'other macrophage']
macrophage.insert(3, "Classes", np.select(conditions, values))
macrophage.to_csv('macrophage.csv')


In [16]:
wf = df[['geo_accession', 'platform_id', 'source_name_ch1']] #wf - working frame
T_cells= wf[wf['source_name_ch1'].str.contains('/sT[-,\s)]cell[\s,s]|\sCD8[+,\s][\s]|[\s]CD4[+,\s][\s]', case=False)]
T_cells.insert(2, "Cell_type", 'T_cells')
conditions = [
    (T_cells['source_name_ch1'].str.contains('[\s]CD8[+,\s][\s]',case=False, regex=True)),
    (T_cells['source_name_ch1'].str.contains('[\s]CD4[+,\s][\s]',case=False, regex=True)),
    (T_cells['source_name_ch1'].str.contains('YFP+', case=False, regex=True)),
    (T_cells['source_name_ch1'].str.contains('V\xce\xb34', case=False)),
    (T_cells['source_name_ch1'].str.contains('Tumor-infiltrating', case=False)),
    (T_cells['source_name_ch1'].str.contains('Thymus|Thymic', case=False)),
    (T_cells['source_name_ch1'].str.contains('Splenic|spleen', case=False)),
    (~T_cells['source_name_ch1'].str.contains('[\s]CD8[+,\s]|[\s]CD4[+,\s][\s]|YFP+|V\xce\xb34|Tumor-infiltrating|Thymus|Thymic|Splenic|spleen', case=False, regex=True))
]
values = ['CD8+ T cells', 'CD4+ T cells', 'YFP+ T cells','V\xce\xb34 T cell', 'Tumor-infiltrating T cells', 'Thymus T cells', 'Splenic T cells', 'other T cells']
T_cells.insert(3, "Classes", np.select(conditions, values))
T_cells.to_csv('T_cells.csv')

In [17]:
wf = df[['geo_accession', 'platform_id', 'source_name_ch1']] #wf - working frame
B_cells= wf[wf['source_name_ch1'].str.contains('[\s]B[-, \s]cell[s,\s]', case=False, regex=True)]
B_cells.insert(2, "Cell_type", 'B_cells')
conditions = [
    (B_cells['source_name_ch1'].str.contains('splenic|from spleen|spleenic',case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Germinal center',case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Marginal zone' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Follicular|folicular' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('B220+' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('Primary' , case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('NP+ GC', case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('immature', case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('CH12 ', case=False, regex=True)),
    (B_cells['source_name_ch1'].str.contains('CD19+', case=False, regex=True)),
    (~B_cells['source_name_ch1'].str.contains('splenic|from spleen|spleenic|Germinal center|Marginal zone|Follicular|folicular|B220+|primary|NP+ GC|immature|CH12|CD19+ ' , case=False, regex=True))

]
values = ['splenic B cells', 'Germinal center B cells', 'Marginal zone B cells', 'Follicular B cells', 'B220+ B cells','Primary B cells','NP+ GC B cells','Immature B cells','CH12 line','CD19+ B cells', 'other B-cells']
B_cells.insert(3, "Classes", np.select(conditions, values))
B_cells.to_csv('B_cells.csv')

In [18]:
B_cells=pd.read_csv('B_cells.csv')
T_cells=pd.read_csv('T_cells.csv')
macrophage=pd.read_csv('macrophage.csv')
great_table = pd.concat([B_cells, T_cells, macrophage], axis=0)
great_table.to_csv('great_table.csv')

get indicis of lines with geo accession
indices[i]=indices[i-1] можно так 

In [19]:
great_table=pd.read_csv('great_table.csv')
df = pd.read_csv('table.csv')
res=df[df.geo_accession.isin(great_table.geo_accession)].reset_index(drop=False)
indices=res['index']
indices2=[]
for i in indices:
    f=i
    indices2.append(f)

In [317]:
print(len(great_table))
print(len(indices))

9410
9410


create an expression table
'''
'''
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
   data_set = f['data']['expression']
   data=data_set[::,indices2]
   expression = pd.DataFrame(data, columns=indices2)
   expression.to_csv('expression.csv')
   f.close()

In [318]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
   data_set = f['data']['expression']
   data=data_set[::,indices2]
   expression = pd.DataFrame(data, columns=indices)
   expression.to_csv('expression3.csv')
   f.close()

In [319]:
print(len(expression))
print(expression.shape[1])

32544
9410


Удалим из большой таблицы все буквы б из всех строк таблицы


In [235]:
```
#df = pd.read_csv('great_table.csv')  
#df = df.replace(to_replace ="[b']", value = '', regex = True)
#df.to_csv('great_table2.csv')     
#df = pd.read_csv('table.csv')  
#df = df.replace(to_replace ="[b']", value = '', regex = True)
#df.to_csv('table.csv')    
```

Заменим индексы колонок в таблице экспрессии на gsm
GSM1849922 - должно быть в позиции 28 - и это верно

In [320]:
wf=pd.read_csv('table.csv')
#df.drop(df.columns[[0]], axis = 1)
expression.columns = wf.loc[indices]['geo_accession']
print(expression.head())
expression.to_csv('expression4.csv')   

geo_accession  GSM1356226  GSM1849922  GSM1897413  GSM1262088  GSM1119597  \
0                    3228        1274         114         810         440   
1                    1185         437          12          56         151   
2                     114          35          21          11         138   
3                    3548        3126          95         520         254   
4                    4075        1033          40         157         465   

geo_accession  GSM1533925  GSM1511235  GSM1808311  GSM1823626  GSM1713923  \
0                     117         500        1425         326        1121   
1                      78         136         757         237         200   
2                       4          46         101          15          44   
3                     112        1954        1063         558        1140   
4                     261          54        1834         948         441   

geo_accession  ...  GSM5550906  GSM5550908  GSM5550909  GSM5558220  \
0   

In [321]:
print(indices.head())
print(indices2[0:5])

0    19
1    28
2    39
3    65
4    77
Name: index, dtype: int64
[19, 28, 39, 65, 77]


In [322]:
df = pd.read_csv('table.csv')  
print(df.iloc[28])


Unnamed: 0                                                             28
taxid_ch1                                                        b'10090'
status                                           b'Public on Jul 15 2016'
source_name_ch1                                            b'Pro B cells'
series_id                                           b'GSE72018\tGSE72019'
relation                b'BioSample: https://www.ncbi.nlm.nih.gov/bios...
readstotal                                                       50911147
readsaligned                                                     45076587
platform_id                                                      GPL13112
organism_ch1                                              b'Mus musculus'
molecule_ch1                                                 b'total RNA'
library_strategy                                               b'RNA-Seq'
library_selection                                                 b'cDNA'
library_source                        

Теперь строки в таблице с экспрессией номер 4 нужно заменить названия строк на гены - indices 2

In [323]:
with h5py.File('mouse_matrix_v11.h5', 'r') as f:
   data_set = f['meta']['genes']['genes']
   data=data_set[::]
   names_lines = pd.DataFrame(data)
   ensembl_set=f['meta']['genes']['ensembl_gene_id']
   ensembl=ensembl_set[::]
   ensemble = pd.DataFrame(ensembl)    
   f.close()
print(names_lines.head())
print(ensemble.head())
print(len(ensemble))
print(len(names_lines))


                  0
0  b'0610007P14Rik'
1  b'0610009B22Rik'
2  b'0610009L18Rik'
3  b'0610009O20Rik'
4  b'0610010F05Rik'
                       0
0             b'missing'
1  b'ENSMUSG00000007777'
2  b'ENSMUSG00000043644'
3             b'missing'
4  b'ENSMUSG00000042208'
32544
32544


In [324]:
expression=pd.read_csv('expression4.csv')
expression2 = pd.concat([ensemble, names_lines, expression], axis=1)
expression2.pop('Unnamed: 0')
expression2.to_csv('expression5.csv')   
print(expression2.head())


                       0                 0  GSM1356226  GSM1849922  \
0             b'missing'  b'0610007P14Rik'        3228        1274   
1  b'ENSMUSG00000007777'  b'0610009B22Rik'        1185         437   
2  b'ENSMUSG00000043644'  b'0610009L18Rik'         114          35   
3             b'missing'  b'0610009O20Rik'        3548        3126   
4  b'ENSMUSG00000042208'  b'0610010F05Rik'        4075        1033   

   GSM1897413  GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  \
0         114         810         440         117         500        1425   
1          12          56         151          78         136         757   
2          21          11         138           4          46         101   
3          95         520         254         112        1954        1063   
4          40         157         465         261          54        1834   

   ...  GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  \
0  ...        1629        1079         839    

In [20]:
d = pd.read_csv('great_table.csv')  
d = d.replace(to_replace ="[b']", value = '', regex = True)
d.to_csv('great_table2.csv')    
f = pd.read_csv('expression5.csv')  
f.pop('Unnamed: 0')
f.rename(columns={'0': "ensemble", '0.1': "symbol"}, inplace=True)
f.to_csv('expression6.csv')   


In [326]:
print(d.head())

   Unnamed: 0.1  Unnamed: 0 geo_accession platform_id Cell type  \
0             0          28    GSM1849922    GPL13112   B_cells   
1             1          38    GSM1897413    GPL13112   B_cells   
2             2         334    GSM1849954    GPL13112   B_cells   
3             3         341    GSM1229014    GPL13112   B_cells   
4             4         570    GSM1234739    GPL13112   B_cells   

              Classes     source_name_ch1  
0       other B-cells         Pro B cells  
1  Follicular B cells  Follicular B cells  
2       other B-cells         B1A B cells  
3       other B-cells     Dnmt-KO B-Cells  
4     Primary B cells     Primary B-cells  


In [327]:

print(f.head())

             ensemble         symbol  GSM1356226  GSM1849922  GSM1897413  \
0             missing  0610007P14Rik        3228        1274         114   
1  ENSMUSG00000007777  0610009B22Rik        1185         437          12   
2  ENSMUSG00000043644  0610009L18Rik         114          35          21   
3             missing  0610009O20Rik        3548        3126          95   
4  ENSMUSG00000042208  0610010F05Rik        4075        1033          40   

   GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  ...  \
0         810         440         117         500        1425  ...   
1          56         151          78         136         757  ...   
2          11         138           4          46         101  ...   
3         520         254         112        1954        1063  ...   
4         157         465         261          54        1834  ...   

   GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  GSM5558224  \
0        1629        1079         839        

 ### Создать таблицу - перечение с той что пришлют

In [20]:
dee2_exp=pd.read_csv('dee2_exp.csv')
print(dee2_exp.head())

   GSM4042993  GSM4042994  GSM4042995  GSM4042996  GSM4042997  GSM4042998  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           0           0           0           0           0   
4           0           0           0           0           0           0   

   GSM4042999  GSM4043000  GSM4043001  GSM4043002  ...  GSM1234735  \
0           0           0           0           0  ...           0   
1           0           0           0           0  ...           1   
2           0           0           0           0  ...           0   
3           0           1           0           0  ...           0   
4           0           0           0           0  ...           0   

   GSM1234736  GSM1234737  GSM1234738  GSM1234739  GSM1234740  GSM1234741  \
0           0           0           0  

In [359]:
dee2_genes=pd.read_csv('dee2_genes.csv')
print(dee2_genes.head())
print(dee2_genes.shape[0])
print(dee2_exp.shape[1])

             ensemble         symbol
0  ENSMUSG00000102693  4933401J01Rik
1  ENSMUSG00000064842        Gm26206
2  ENSMUSG00000051951           Xkr4
3  ENSMUSG00000102851        Gm18956
4  ENSMUSG00000103377        Gm37180
52636
5558


In [22]:
dee2 = pd.concat([dee2_genes, dee2_exp], axis=1)
dee2.to_csv('dee.csv')   
print(dee2.head())


             ensemble         symbol  GSM4042993  GSM4042994  GSM4042995  \
0  ENSMUSG00000102693  4933401J01Rik           0           0           0   
1  ENSMUSG00000064842        Gm26206           0           0           0   
2  ENSMUSG00000051951           Xkr4           0           0           0   
3  ENSMUSG00000102851        Gm18956           0           0           0   
4  ENSMUSG00000103377        Gm37180           0           0           0   

   GSM4042996  GSM4042997  GSM4042998  GSM4042999  GSM4043000  ...  \
0           0           0           0           0           0  ...   
1           0           0           0           0           0  ...   
2           0           0           0           0           0  ...   
3           0           0           0           0           1  ...   
4           0           0           0           0           0  ...   

   GSM1234735  GSM1234736  GSM1234737  GSM1234738  GSM1234739  GSM1234740  \
0           0           0           0        

#### Проведем сортировку

In [4]:
exp = pd.read_csv('expression6.csv', encoding="utf8") # большая таблица
dee = pd.read_csv('dee.csv') #маленькая таблица
exp.pop('Unnamed: 0')
dee.pop('Unnamed: 0')
exp=exp.sort_values(by=['ensemble','symbol'])
dee=dee.sort_values(by=['ensemble','symbol'])
print(exp.head())
print(exp.shape[0])
print(dee.head())

                 ensemble symbol  GSM1356226  GSM1849922  GSM1897413  \
16843  ENSMUSG00000000001  Gnai3       17328        8597         507   
23599  ENSMUSG00000000003    Psn           0           0           0   
3402   ENSMUSG00000000028  Cdc45        1577        5479          27   
26959  ENSMUSG00000000037  Scml2          16          86           0   
1743   ENSMUSG00000000049   Apoh           1           0           0   

       GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  ...  \
16843        6175         536        1079        7421       10601  ...   
23599           0           0           0           0           0  ...   
3402           53         293          14         187         512  ...   
26959           7          50           0          10          32  ...   
1743            0           0           0           0           0  ...   

       GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  GSM5558224  \
16843       23609       13185       25213

In [5]:
print(exp.shape[0])
print(exp.shape[1])
print(dee.shape[0])
print(dee.shape[1])

32544
9412
52636
5560


#### Из большой таблицы выделить ту часть, что соответствует маленькой

In [6]:
dee_cross=dee[dee.symbol.isin(exp.symbol)|dee.ensemble.isin(exp.ensemble)  ]
print(dee_cross.head())
print(dee_cross.shape[0])

                 ensemble symbol  GSM4042993  GSM4042994  GSM4042995  \
11893  ENSMUSG00000000001  Gnai3        3962        3807        2487   
8469   ENSMUSG00000000003   Pbsn           0           0           0   
46315  ENSMUSG00000000028  Cdc45        1254        1209        1144   
9726   ENSMUSG00000000037  Scml2           0           0           1   
39400  ENSMUSG00000000049   Apoh           0           0           3   

       GSM4042996  GSM4042997  GSM4042998  GSM4042999  GSM4043000  ...  \
11893        3122        3196        2420        5688        7300  ...   
8469            0           0           0           0           0  ...   
46315        1362        1310        1264        1109        1569  ...   
9726            1           0           0           2           0  ...   
39400           0           1           4           0           0  ...   

       GSM1234735  GSM1234736  GSM1234737  GSM1234738  GSM1234739  GSM1234740  \
11893        4077        4671        7293

In [7]:
exp_cross=exp[exp.symbol.isin(dee_cross.symbol)|exp.ensemble.isin(dee_cross.ensemble)  ]
print(exp_cross.head())
print(exp_cross.shape[0])

                 ensemble symbol  GSM1356226  GSM1849922  GSM1897413  \
16843  ENSMUSG00000000001  Gnai3       17328        8597         507   
23599  ENSMUSG00000000003    Psn           0           0           0   
3402   ENSMUSG00000000028  Cdc45        1577        5479          27   
26959  ENSMUSG00000000037  Scml2          16          86           0   
1743   ENSMUSG00000000049   Apoh           1           0           0   

       GSM1262088  GSM1119597  GSM1533925  GSM1511235  GSM1808311  ...  \
16843        6175         536        1079        7421       10601  ...   
23599           0           0           0           0           0  ...   
3402           53         293          14         187         512  ...   
26959           7          50           0          10          32  ...   
1743            0           0           0           0           0  ...   

       GSM5550906  GSM5550908  GSM5550909  GSM5558220  GSM5558221  GSM5558224  \
16843       23609       13185       25213

1. Отсортировать положение столбцов
2. Перекрестить таблицы по списку столбцов

In [8]:
exp_cross = exp_cross[sorted(exp_cross.columns )]
dee_cross = dee_cross[sorted(dee_cross.columns )]
print(exp_cross.head())
print(dee_cross.head())

       GSM1026888  GSM1026889  GSM1026890  GSM1026891  GSM1026892  GSM1026893  \
16843        5567        5857        4467        5325        6059        5579   
23599           0           0           0           0           0           0   
3402           16          61          12          85         124         172   
26959           2           6           5           0           3          12   
1743            2           0           1           1           0           0   

       GSM1026894  GSM1026895  GSM1026896  GSM1026897  ...  GSM994541  \
16843        4753        5581        4557        3708  ...       9219   
23599           0           0           0           0  ...          0   
3402          117         188           7          29  ...       1913   
26959          10          16           1           3  ...         26   
1743            2           0           3           1  ...          0   

       GSM994542  GSM994543  GSM994544  GSM994545  GSM994546  GSM994547  \

In [9]:
col=dee_cross.columns
g=[]
for i in col:
    g.append(i)

In [10]:
exp_cross=exp_cross.loc[:, exp_cross.columns.intersection(g)]
col=exp_cross.columns
h=[]
for i in col:
    h.append(i)
dee_cross=dee_cross.loc[:, dee_cross.columns.intersection(h)]

In [11]:
print(exp_cross.head())
print(dee_cross.head())

       GSM1026888  GSM1026889  GSM1026890  GSM1026891  GSM1026892  GSM1026893  \
16843        5567        5857        4467        5325        6059        5579   
23599           0           0           0           0           0           0   
3402           16          61          12          85         124         172   
26959           2           6           5           0           3          12   
1743            2           0           1           1           0           0   

       GSM1026894  GSM1026895  GSM1026896  GSM1026897  ...  GSM951562  \
16843        4753        5581        4557        3708  ...       1090   
23599           0           0           0           0  ...          0   
3402          117         188           7          29  ...        123   
26959          10          16           1           3  ...          4   
1743            2           0           3           1  ...          0   

       GSM951563  GSM951564  GSM951565  GSM951566  GSM951567  GSM951568  \

In [12]:
exp_cross.to_csv('exp_cross.csv')
dee_cross.to_csv('dee_cross.csv')